Import libraries

In [1]:
import pandas as pd
import numpy as np
import re
import time
import seaborn
import requests
!pip install pytrends
import pytrends
from pytrends.request import TrendReq
from bs4 import BeautifulSoup, Comment
from urllib.parse import urlparse

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Cache-Control': 'no-cache',
    'Pragma': 'no-cache'
}

Read the list of keyword (domain of 35000 companies)

In [6]:
path = r'C:\Users\AshleyNguyen\Desktop\script\interest_df\\'
df = pd.read_csv(path + 'test2.csv')

Import the dictionary from nltk to check for common words

In [8]:
import nltk
nltk.download('words')
from nltk.corpus import words
"equip" in words.words()

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\AshleyNguyen\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

Preprocessing functions

In [9]:
def strip_www(domain):
    if type(domain) == str:
        if domain.startswith("ww"):
            domain = domain.split('.')[1]
        else:
            domain = domain.split('.')[0]
    return domain

In [10]:
def find_keyword(s):
    if type(s) == str:
        if strip_www(s) in words.words():
            keyword = re.sub('^w{0,3}\d*\.','',s)
        else:
            keyword = strip_www(s)
    else:
        keyword = ''
    return keyword

In [11]:
def get_interest_over_time(keyword):
    """
    Fetches the interest over time data for the given keyword from Google Trends.

    Args:
        keyword (str): The keyword to retrieve the interest data for.

    Returns:
        list: A list of interest over time data for the specified keyword.
    """
    pytrends = TrendReq(hl='en-US', tz=360, timeout=(10,25), retries=15, backoff_factor=0.1)
    pytrends.build_payload([keyword], timeframe='today 12-m')
    interest_over_time = pytrends.interest_over_time()
    
    if 'isPartial' in interest_over_time.columns:
        interest_over_time = interest_over_time.drop(columns='isPartial')
    
    return interest_over_time.T

Main function to get the interest over time

In [13]:
from tqdm import tqdm

interest_df = pd.DataFrame()
# progress_bar = tqdm(range(len(df['keyword'].tolist())))

num_companies =  len(df['0'].tolist())
list_companies = df['0'].tolist()

stop_position = 19310
progress_bar = tqdm(range(stop_position, num_companies))
i = stop_position

for i in progress_bar:
    progress_bar.set_description(f"Processing keyword {i+1}/{num_companies - stop_position}")
    
    if df['0'].tolist()[i] != '':
        interest_df_1 = get_interest_over_time(list_companies[i])
    else:
        interest_df_1 = pd.DataFrame()
    
    interest_df = pd.concat([interest_df, interest_df_1], axis=0)
    
interest_df.reset_index(drop=True, inplace=True)

Processing keyword 20912/15858:  10%|█         | 1601/15858 [37:35<5:34:49,  1.41s/it]  


KeyboardInterrupt: 

In [14]:
interest_df

date,2022-06-12,2022-06-19,2022-06-26,2022-07-03,2022-07-10,2022-07-17,2022-07-24,2022-07-31,2022-08-07,2022-08-14,...,2023-03-26,2023-04-02,2023-04-09,2023-04-16,2023-04-23,2023-04-30,2023-05-07,2023-05-14,2023-05-21,2023-05-28
carwow,86,88,86,83,83,74,76,81,90,83,...,85,89,84,90,84,78,80,80,84,74
theriveter,0,0,0,0,0,0,0,64,0,0,...,0,0,0,0,0,0,0,0,0,0
brilliant.tech,0,83,0,0,0,0,0,0,0,0,...,0,0,0,0,97,0,0,76,49,0
alison.com,25,38,56,65,32,58,53,74,44,38,...,68,66,56,45,48,28,61,50,85,21
vividcortex,0,0,0,0,0,0,0,24,0,0,...,0,0,27,73,0,0,0,51,0,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wemoney,36,39,17,18,100,93,55,38,39,0,...,59,38,0,18,23,27,0,16,0,25
beautiful.ai,10,21,24,11,15,9,13,26,11,25,...,61,60,67,68,100,74,72,72,64,58
nivaura,0,0,0,0,0,38,0,0,0,0,...,0,0,0,0,0,0,0,0,33,36
respondology,86,0,100,0,0,51,0,0,0,0,...,0,0,86,0,66,0,87,0,0,0


Write to file

In [15]:
interest_df.to_csv(path+'interest_df_19310.csv')